In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

%run LSYM_ABBA_QuPath.py

In [ ]:
#------ USER INPUT REQUIRED -------

# Please set the string variables below to specify the input and output data folders:

# Location of the ABBA state .json file:
#ABBA_json="D:\\Alex\\EPFL\\Data\\Confocal\\Nov03_Nov0521_MeA_VGATCre_synaptophysin_Aiste\\FV6570\\ABBA\\FV6570_ABBA-new.json"
ABBA_json="D:\\Alex\\EPFL\\Papers\\Paper_AStria_Michael\\J30063_D1R_tdT\\ABBA\\J30063_ABBA-new-thickness-fill-gap.json"

# Please specify the folder name for creating the output sub-folders:
#data_path="D:\\Alex\\EPFL\\Data\\Confocal\\Nov03_Nov0521_MeA_VGATCre_synaptophysin_Aiste\\FV6570\\test\\"
data_path="D:\\Alex\\EPFL\\Papers\\Paper_AStria_Michael\\J30063_D1R_tdT\\"

# Sub-folder names for the extracted original data and for the processed summary data:
path_prefix=data_path+"input_dataset\\"
path_prefix_results=data_path+"results\\"

# Custom prefix for the summary data files:
#result_filename="FV6570_"
result_filename="J30063_"

In [ ]:
# parsing the ABBA project and creating the output subfolders...
df_atlas, df_index = create_subfolders(ABBA_json, path_prefix, path_prefix_results, result_filename)

In [ ]:
# Please use automatically created index file as a template to modify the list of images for processing
# AP coordinates of slices in the index file are approximate; these will be re-evaluated later on 

# An optional boolean column "Swap_sides" (possible values: True or False), together with the string column "Swap_node" 
# (the latter expected only if the Swap_sides flag is True), can be used to force swap left and right sides of the atlas sub-tree.
# If both columns are present, and if "Swap_sides" is True for a given image, the whole sub-tree of the atlas
# below the specified node will be swapped between the left and the right sides. 
# This could be is useful when two parts of the brain (e.g. detached posterior parts of cortex)
# were accidentally swapped during mounting procedure.
# The swap operation will be logged by setting a flag "swapped_sides_flag" to 1 in the combined output data file.

# Name of the working index file, optionally edited as described above (location defined by path_prefix variable):
index_filename="J30063_autoindex_test.xlsx"
#index_filename="FV6570_autoindex_test.xlsx"


In [ ]:
# loading the working index file

df_index, swap_pos = read_index_file(index_filename, path_prefix)

In [ ]:
# Importing the data from QuPath
# Individual .csv files with annotations and detections will be created for every image

# ! This step may take time and does NOT have to be repeated if it was done previously
# That is, if corresponding .csv file already exist, it is recommended to skip to the next step 

idx_files_list, dets_files_list = extract_QuPath_data(df_index, path_prefix)

In [ ]:
# loading the annotation and detection data files and pre-processing

df, df_dets = load_csv_data(df_index, swap_pos, path_prefix, path_prefix_results, result_filename, df_atlas)

In [ ]:
#  File with the "clean" detection coordinates can be e.g.used directly to plot in Brainrender package. 
#  For details, see: Claudi et al., 2020. “Brainrender. A Python Based Software for Visualisation of Neuroanatomical and Morphological Data.” 
#  Cold Spring Harbor Laboratory. https://doi.org/10.1101/2020.02.23.961748
#  https://github.com/brainglobe/brainrender

In [ ]:
# List of column names to analyse
list_to_analyze=["Num Detections", "Area µm^2", "Detection_Density"]

# Splitting the data into the left and the right sub-trees, collating the data and saving the intermediate results   
# The output is a list of 6 dataframe objects that can be used later on:
# [df_left, df_left_tree, df_left_collated, df_right, df_right_tree, df_right_collated]

df_list = process_left_right_trees(df, list_to_analyze, path_prefix_results, result_filename)

In [ ]:
#lst_right=[]
#lst_left=[]

## -> CHOOSE the output file format
save_mode="csv" 
#save_mode="xlsx"


## -> Below, please uncomment/comment the blocks of commands to CHOOSE ONE of filters for the output summary data
## -> The filter type is set by the value of "mode" variable and a few associated variables
## -> customizable for a desired content of the summary

## summarize for a desired hierarchy level (defined by -mode) of the atlas tree

#mode=-3 
#super_list=["root"]
#term_flag=False
#file_suffix="level"+str(-mode)

## OR summarize for all the terminal leaves of the atlas tree

#mode=1  
#super_list=["root"]
#term_flag=True
#file_suffix="whole_tree"

## OR summarize for the whole atlas tree

mode=2 
super_list=["root"]
term_flag=False
file_suffix="whole"

## OR summarize for the ROIs defined with a list of Allen IDs

#mode=4 
#super_list=["root"]
#term_flag=False
#lst_right=[806, 654, 182305701]
#lst_left=[806, 654, 182305701]
#file_suffix="IDs"

## OR summarize for the terminal leaves descending from selected parents listed as acronyms

#mode=3 
#term_flag=True
#super_list=["Isocortex"]#["TH","HY","PAL","STR","CNU","CTXsp","HPF","OLF","Isocortex"]
#file_suffix = "" 


In [ ]:
left_df_list, right_df_list = summary_per_ROI(df_list, super_list, mode, save_mode, term_flag, file_suffix, path_prefix_results, result_filename)

In [ ]:
# The main script is finished, all results were saved in the output files.
# The files can be further processed to sort and plot density distributions per brain area or 
# to plot the spatial AP profiles of detectons.

# For these purposes, scripts for IgorPro 7 (Wavemetrics Inc) are currently available, 
# and those scripts also support averaging results across different brains.


# In the following cells, some example visualizations are performed to briefly check the data.
# Horizontal bar graphs and AP-distribution graphs can be generated by IgorPro scripts.


In [ ]:
# Below there is a description of the list of pandas dataframes that remain in memory after storing the .csv files 
# and which are used to plot the data in following cells.

# left_df_list, right_df_list: lists of dataframe objects 
# [dets_vsAP, dens_vsAP, area_vsAP, dets_total, dens_total, area_total]
# with the information on detections, densities, ROI areas as functions of AP coordinates and respective total values
# for the left and the right sides (all stored in respective .csv files)

# df_list: list of dataframes objects
# [df_left, df_left_tree, df_left_collated, df_right, df_right_tree, df_right_collated]
# containg the global information, hyerarchical tree information and collated tree data 
# for the left and the right sides (all stored in respective .csv files)

In [ ]:
# interactive hierarchy plot for the LEFT-side tree results
tree_path=["level_0"]
for i in range(np.max(df_list[1].loc[:,"Tree_level"])):
    tree_path.append("level_"+str(i+1))

fig2 = px.sunburst(df_list[1], path=tree_path, values='Num Detections', color='level_4', branchvalues="remainder", width=1000, height=1000)
fig2.update_layout(title_text="Sunburst diagram for the LEFT-side tree (total counts across all AP levels)", font_size=9)
fig2.show()

In [ ]:
# interactive hierarchy plot for the RIGHT-side tree results
tree_path=["level_0"]
for i in range(np.max(df_list[4].loc[:,"Tree_level"])):
    tree_path.append("level_"+str(i+1))

fig1 = px.sunburst(df_list[4], path=tree_path, values='Num Detections', color='level_4', branchvalues="remainder", width=1000, height=1000)
fig1.update_layout(title_text="Sunburst diagram for the RIGHT-side tree (total counts across all AP levels)", font_size=9)
fig1.show()


In [ ]:
# Plotting the total number of detections in brain regions across all the slices
# This can be useful for a single group of brain areas after running the last step of analysis using
# a single hub structure as a list, for example:
## mode=3 
## term_flag=True
## super_list=["Isocortex"]
## file_suffix = ""

# Sorting by the left side values? If False, will be sorted by Right side values
sorting_left = False

df_tmp_left=left_df_list[3].drop("Root_Atlas_AP", axis=1)
df_tmp_right=right_df_list[3].drop("Root_Atlas_AP", axis=1)
df_tmp=pd.concat([df_tmp_left, df_tmp_right], ignore_index=True, sort=False).fillna(0)
if (sorting_left):
    sorted_df=df_tmp.sort_values(df_tmp.first_valid_index(), axis=1, ascending=False)
else:
    sorted_df=df_tmp.sort_values(df_tmp.last_valid_index(), axis=1, ascending=False)
        
brain_areas=sorted_df.columns.str.split(";")
legends=[sorted_df.columns.str.split(";")[i][1]+" ("+sorted_df.columns.str.split(";")[i][0]+")" for i in range(len(brain_areas))]

dets_left=sorted_df.to_numpy()[0]*(-1)
dets_right=sorted_df.to_numpy()[1]


fig, ax = plt.subplots(figsize=(4, len(brain_areas)/8))

y_pos = np.arange(len(legends))

ax.barh(y_pos, dets_left, height=0.8, align='center')
ax.barh(y_pos, dets_right, height=0.8, align='center')
ax.set_ylim(-0.5, len(legends)-0.5)
ax.set_yticks(y_pos, labels=legends)
ax.yaxis.set_tick_params(labelsize=7)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top') 
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Detections (#)')
ax.set_title('Total number of detections across analyzed slices')

plt.gca().legend(["Left side", "Right side"], bbox_to_anchor=(1.1, 0, 0.4, 1), loc='upper left', ncol=1, mode="expand", borderaxespad=0.2)
plt.show()



In [ ]:
# Plotting the spatial AP-distribution of detections for the N_top brain areas from the previous step 

# Please set the number of brain areas to plot:
N_top=10

areas2plot=sorted_df.columns[0:N_top]
acronyms2plot=areas2plot.str.split(";")
# if str(left_df_list[0].loc[j,i])!='nan'
y_dets_left = [[left_df_list[0].loc[j,i] for i in areas2plot] for j in range(left_df_list[0].shape[0])]
y_dets_right = [[right_df_list[0].loc[j,i] for i in areas2plot] for j in range(right_df_list[0].shape[0])]
x_dets_l = [[float(j) for j in df_list[2].where(df_list[2]["Class"]==i[0]).dropna().reset_index().loc[0,"ROI_Atlas_AP"].split(";")] for i in acronyms2plot]
x_dets_r = [[float(j) for j in df_list[5].where(df_list[5]["Class"]==i[0]).dropna().reset_index().loc[0,"ROI_Atlas_AP"].split(";")] for i in acronyms2plot]

cntr=np.zeros(N_top, dtype=int)
x_dets_left=[x[:] for x in y_dets_left]

for i in range(len(y_dets_left)):
    for j in range(N_top):
        if (str(y_dets_left[i][j])!='nan'):
            x_dets_left[i][j]=x_dets_l[j][cntr[j]]
            cntr[j]+=1
            
cntr=np.zeros(N_top, dtype=int)
x_dets_right=[x[:] for x in y_dets_right]

for i in range(len(y_dets_right)):
    for j in range(N_top):
        if (str(y_dets_right[i][j])!='nan'):
            x_dets_right[i][j]=x_dets_r[j][cntr[j]]
            cntr[j]+=1


legends=[i[0] for i in acronyms2plot]

fig=plt.figure()
dets_l = fig.add_axes([0.15, 0.1, 1, 1])
dets_l.set_xlabel("AP coordinate (mm)")
dets_l.set_ylabel("Detections, LEFT side")
dets_l.plot(x_dets_left,y_dets_left)

dets_r = fig.add_axes([1.4, 0.1, 1, 1])
dets_r.set_xlabel("AP coordinate (mm)")
dets_r.set_ylabel("Detections, RIGHT side")
dets_r.plot(x_dets_right,y_dets_right)


plt.gca().legend(legends, bbox_to_anchor=(1.1, 0, 0.35, .45), loc='upper left', ncol=1, mode="expand", borderaxespad=0)

plt.show()

In [ ]:
# Plotting the average density of detections in brain regions across all the slices
# This can be useful for a single group of brain areas after running the last step of analysis using
# a single hub structure as a list, for example:
## mode=3 
## term_flag=True
## super_list=["Isocortex"]
## file_suffix = ""

# Sorting by the left side values? If False, will be sorted by Right side values
sorting_left = True


df_tmp_left=left_df_list[4].drop("Root_Atlas_AP", axis=1)
df_tmp_right=right_df_list[4].drop("Root_Atlas_AP", axis=1)
df_tmp=pd.concat([df_tmp_left, df_tmp_right], ignore_index=True, sort=False).fillna(0)
if (sorting_left):
    sorted_df=df_tmp.sort_values(df_tmp.first_valid_index(), axis=1, ascending=False)
else:
    sorted_df=df_tmp.sort_values(df_tmp.last_valid_index(), axis=1, ascending=False)
        
brain_areas=sorted_df.columns.str.split(";")
legends=[sorted_df.columns.str.split(";")[i][1]+" ("+sorted_df.columns.str.split(";")[i][0]+")" for i in range(len(brain_areas))]

dets_left=sorted_df.to_numpy()[0]*(-1)
dets_right=sorted_df.to_numpy()[1]


fig, ax = plt.subplots(figsize=(4, len(brain_areas)/8))

y_pos = np.arange(len(legends))

ax.barh(y_pos, dets_left, height=0.8, align='center')
ax.barh(y_pos, dets_right, height=0.8, align='center')
ax.set_ylim(-0.5, len(legends)-0.5)
ax.set_yticks(y_pos, labels=legends)
ax.yaxis.set_tick_params(labelsize=7)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top') 
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Density of detections (1/mm$^2$)')
ax.set_title('Average density of detections across analyzed slices')

plt.gca().legend(["Left side", "Right side"], bbox_to_anchor=(1.1, 0, 0.4, 1), loc='upper left', ncol=1, mode="expand", borderaxespad=0.2)
plt.show()

In [ ]:
# Plotting the spatial AP-distribution of detection densities for the N_top brain areas from the previous step 

# Please set the number of brain areas to plot:
N_top=10

areas2plot=sorted_df.columns[0:N_top]
acronyms2plot=areas2plot.str.split(";")

y_dens_left = [[left_df_list[1].loc[j,i] for i in areas2plot] for j in range(left_df_list[1].shape[0])]
y_dens_right = [[right_df_list[1].loc[j,i] for i in areas2plot] for j in range(right_df_list[1].shape[0])]
x_dens_l = [[float(j) for j in df_list[2].where(df_list[2]["Class"]==i[0]).dropna().reset_index().loc[0,"ROI_Atlas_AP"].split(";")] for i in acronyms2plot]
x_dens_r = [[float(j) for j in df_list[5].where(df_list[5]["Class"]==i[0]).dropna().reset_index().loc[0,"ROI_Atlas_AP"].split(";")] for i in acronyms2plot]

cntr=np.zeros(N_top, dtype=int)
x_dens_left=[x[:] for x in y_dens_left]

for i in range(len(y_dens_left)):
    for j in range(N_top):
        if (str(y_dens_left[i][j])!='nan'):
            x_dens_left[i][j]=x_dens_l[j][cntr[j]]
            cntr[j]+=1
            
cntr=np.zeros(N_top, dtype=int)
x_dens_right=[x[:] for x in y_dens_right]

for i in range(len(y_dens_right)):
    for j in range(N_top):
        if (str(y_dens_right[i][j])!='nan'):
            x_dens_right[i][j]=x_dens_r[j][cntr[j]]
            cntr[j]+=1

legends=[i[0] for i in acronyms2plot]

fig=plt.figure()
dens_l = fig.add_axes([0.15, 0.1, 1, 1])
dens_l.set_xlabel("AP coordinate (mm)")
dens_l.set_ylabel("Detection density, LEFT side (1/mm$^2$)")
dens_l.plot(x_dens_left,y_dens_left)

dens_r = fig.add_axes([1.4, 0.1, 1, 1])
dens_r.set_xlabel("AP coordinate (mm)")
dens_r.set_ylabel("Detection density, RIGHT side (1/mm$^2$)")
dens_r.plot(x_dens_right,y_dens_right)


plt.gca().legend(legends, bbox_to_anchor=(1.1, 0, 0.35, .45), loc='upper left', ncol=1, mode="expand", borderaxespad=0)

plt.show()